<a href="https://colab.research.google.com/github/mateopolancecofficial/DeepLearningProjects/blob/main/ComputerVision/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import PIL.Image
import tensorflow_datasets as tfds
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [35]:
tf.debugging.set_log_device_placement(True)

if tf.config.list_physical_devices('GPU'):
  if len(tf.config.list_physical_devices('GPU')) > 1:
    strategy = tf.distribute.MirroredStrategy()
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
  else:
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
# download images folder
#!wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
#from shutil import unpack_archive
#unpack_archive('/content/flower_photos.tgz', '/content/')

--2021-06-11 20:41:40--  https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228813984 (218M) [application/x-compressed-tar]
Saving to: ‘flower_photos.tgz’

flower_photos.tgz   100%[===================>] 218.21M   191MB/s    in 1.1s    

2021-06-11 20:41:42 (191 MB/s) - ‘flower_photos.tgz’ saved [228813984/228813984]



In [73]:
# load data parameters
MAX_WIDTH, MAX_HEIGHT = 100, 100
origin = os.path.dirname(os.path.abspath("ImageClassification.ipynb"))
print(origin)
train_path = 'flower_photos'
#test_path = ''
untar = False
color_mode = "rgb"
train_source_folder_path = origin + '/' +  train_path
#test_source_folder_path
print(source_folder_path)

batch_size = 32
validation_split = 0.2
seed = 100


/content
/content/flower_photos


In [63]:
def get_min_img_size(source_folder_path):
  """
  Check if all images are of the same size and get min image width and min image
  height.
  param source_folder_path:      path of a source folder
  return img_width, img_height:  min image width and height scalar values
  """

  img_width, img_height = None, None
  dirs = os.listdir(source_folder_path)

  for dir in dirs:
    files = os.listdir(source_folder_path + "/" + dir)
    for f in files:
      image_path = source_folder_path + "/" + dir + "/" + f
      image = tf.keras.preprocessing.image.load_img(image_path)
      image = keras.preprocessing.image.img_to_array(image)
      if img_width == None:
        img_width, img_height = image.shape[0], image.shape[1]
      else:
        if image.shape[0] < img_width:
          img_width = image.shape[0]
        if image.shape[1] < img_height:
          img_height = image.shape[1]
  
  return img_width, img_height

In [64]:
def set_resize_image_parameters(img_width, img_height):
  """
  Set parameters for resizing input images.
  param img_width, img_height:   image height and image with scalar values
  return img_width, img_height:  resized image height and image with scalar values     
  """

  if img_width > MAX_WIDTH:
    img_width = MAX_WIDTH

  if img_height > MAX_HEIGHT:
    img_height = MAX_HEIGHT
  
  return img_height, img_width



In [74]:
# set tensorflow dataset parameters
# remove unneccessary files from source directory
img_width, img_height = get_min_img_size(train_source_folder_path)
img_width, img_height = set_resize_image_parameters(img_height, img_width)
image_size = (img_width, img_height)

100
100


In [68]:
def get_image_dataset(data_path, color_mode, image_size, subset = None, 
                      validation_split = None, seed = None):
  """
  Create tensorflow dataset from data directory object.
  param data_path:        string, source folder path
  param color_mode:       available color modes are "grayscale", "rgb", "rgba", 
                          (images will be converted to have 1, 3, or 4 channels)
  param image_size:       touple (image_with, image_height)
  param subset:           name of subset ('train', 'validation')
  param validation_split: validation set percentage (float)
  """
    
  if subset:
        validation_split = validation_split
  dataset = tf.keras.preprocessing.image_dataset_from_directory(
      data_path,
      color_mode=color_mode,
      image_size=image_size,
      seed=seed,
      validation_split=validation_split, 
      subset=subset
    )
    
  dataset.class_names.sort()
    
  return {
      "data": dataset.cache().prefetch(
      buffer_size = tf.data.experimental.AUTOTUNE
      ),
      "classNames": dataset.class_names
        }

In [72]:

training_ds = get_image_dataset(
    train_source_folder_path,
    color_mode,
    image_size,
    subset = "training",
    validation_split = validation_split, 
    seed=seed
)

validation_ds = get_image_dataset(
    train_source_folder_path,
    color_mode,
    image_size,
    subset = "validation",
    validation_split = validation_split,
    seed=seed
)

"""
test_ds = get_image_dataset(
    test_source_folder_path,
    color_mode,
    image_size
)
"""

Found 3670 files belonging to 6 classes.
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalAnd in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op SelectV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DummySeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV3 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DummyMemoryCache in device /job:localhost/replica:0/tas